##### Copyright 2020 The OpenFermion Developers

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hamiltonian Time Evolution and Expectation Value Computation

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/openfermion/fqe/tutorials/hamiltonian_time_evolution_and_expectation_estimation"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/OpenFermion/blob/master/docs/fqe/tutorials/hamiltonian_time_evolution_and_expectation_estimation.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/OpenFermion/blob/master/docs/fqe/tutorials/hamiltonian_time_evolution_and_expectation_estimation.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/OpenFermion/docs/fqe/tutorials/hamiltonian_time_evolution_and_expectation_estimation.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

This tutorial describes the FQE's capabilities for Hamiltonian time-evolution and expectation value estimation

Where possible, LiH will be used as an example molecule for the API.

In [ ]:
try:
    import fqe
except ImportError:
    !pip install fqe --quiet

In [ ]:
Print = True
from openfermion import FermionOperator, MolecularData
from openfermion.utils import hermitian_conjugated
import numpy
import fqe

numpy.set_printoptions(floatmode='fixed', precision=6, linewidth=80, suppress=True)
numpy.random.seed(seed=409)

In [ ]:
!curl -O https://raw.githubusercontent.com/quantumlib/OpenFermion-FQE/master/tests/unittest_data/build_lih_data.py

In [ ]:
import build_lih_data

In [ ]:
h1e, h2e, wfn = build_lih_data.build_lih_data('energy')
lih_hamiltonian = fqe.get_restricted_hamiltonian(([h1e, h2e]))
lihwfn = fqe.Wavefunction([[4, 0, 6]])
lihwfn.set_wfn(strategy='from_data', raw_data={(4, 0): wfn})
if Print:
    lihwfn.print_wfn()

## Application of one- and two-body fermionic gates

The API for time propagation can be invoked through the fqe namespace or the wavefunction object

In [ ]:
# dummy geometry
from openfermion.chem.molecular_data import spinorb_from_spatial
from openfermion import jordan_wigner, get_sparse_operator, InteractionOperator, get_fermion_operator

h1s, h2s = spinorb_from_spatial(h1e, numpy.einsum("ijlk", -2 * h2e) * 0.5)
mol = InteractionOperator(0, h1s, h2s)
ham_fop = get_fermion_operator(mol)
ham_mat = get_sparse_operator(jordan_wigner(ham_fop)).toarray()


In [ ]:
from scipy.linalg import expm
time = 0.01
evolved1 = lihwfn.time_evolve(time, lih_hamiltonian)
if Print:
    evolved1.print_wfn()
evolved2 = fqe.time_evolve(lihwfn, time, lih_hamiltonian)
if Print:
    evolved2.print_wfn()
assert numpy.isclose(fqe.vdot(evolved1, evolved2), 1)
cirq_wf = fqe.to_cirq(lihwfn)
evolve_cirq = expm(-1j * time * ham_mat) @ cirq_wf
test_evolve = fqe.from_cirq(evolve_cirq, thresh=1.0E-12)
assert numpy.isclose(fqe.vdot(test_evolve, evolved1), 1)

## Exact evolution implementation of quadratic Hamiltonians

Listed here are examples of evolving the special Hamiltonians.

Diagonal Hamiltonian evolution is supported.

In [ ]:
wfn = fqe.Wavefunction([[4, 2, 4]])
wfn.set_wfn(strategy='random')
if Print:
    wfn.print_wfn()

diagonal = FermionOperator('0^ 0', -2.0) + \
           FermionOperator('1^ 1', -1.7) + \
           FermionOperator('2^ 2', -0.7) + \
           FermionOperator('3^ 3', -0.55) + \
           FermionOperator('4^ 4', -0.1) + \
           FermionOperator('5^ 5', -0.06) + \
           FermionOperator('6^ 6', 0.5) + \
           FermionOperator('7^ 7', 0.3)
if Print:
    print(diagonal)
    
evolved = wfn.time_evolve(time, diagonal)
if Print:
    evolved.print_wfn()

Exact evolution of dense quadratic hamiltonians is supported.  Here is an evolution example using a spin restricted Hamiltonian on a number and spin conserving wavefunction

In [ ]:
norb = 4 
h1e = numpy.zeros((norb, norb), dtype=numpy.complex128) 
for i in range(norb): 
    for j in range(norb): 
        h1e[i, j] += (i+j) * 0.02 
    h1e[i, i] += i * 2.0 

hamil = fqe.get_restricted_hamiltonian((h1e,)) 
wfn = fqe.Wavefunction([[4, 0, norb]]) 
wfn.set_wfn(strategy='random') 
initial_energy = wfn.expectationValue(hamil) 
print('Initial Energy: {}'.format(initial_energy))
evolved = wfn.time_evolve(time, hamil) 
final_energy = evolved.expectationValue(hamil)
print('Final Energy:   {}'.format(final_energy))

The GSO Hamiltonian is for evolution of quadratic hamiltonians that are spin broken and number conserving.

In [ ]:
norb = 4 
h1e = numpy.zeros((2*norb, 2*norb), dtype=numpy.complex128) 
for i in range(2*norb): 
    for j in range(2*norb): 
        h1e[i, j] += (i+j) * 0.02 
    h1e[i, i] += i * 2.0 

hamil = fqe.get_gso_hamiltonian((h1e,)) 
wfn = fqe.get_number_conserving_wavefunction(4, norb) 
wfn.set_wfn(strategy='random') 
initial_energy = wfn.expectationValue(hamil) 
print('Initial Energy: {}'.format(initial_energy))
evolved = wfn.time_evolve(time, hamil) 
final_energy = evolved.expectationValue(hamil)
print('Final Energy:   {}'.format(final_energy))

The BCS hamiltonian evovles spin conserved and number broken wavefunctions.

In [ ]:
norb = 4
time = 0.001
wfn_spin = fqe.get_spin_conserving_wavefunction(2, norb)
hamil = FermionOperator('', 6.0)
for i in range(0, 2*norb, 2):
    for j in range(0, 2*norb, 2):
        opstring = str(i) + ' ' + str(j + 1)
        hamil += FermionOperator(opstring, (i+1 + j*2)*0.1 - (i+1 + 2*(j + 1))*0.1j)
        opstring = str(i) + '^ ' + str(j + 1) + '^ '
        hamil += FermionOperator(opstring, (i+1 + j)*0.1 + (i+1 + j)*0.1j)
h_noncon = (hamil + hermitian_conjugated(hamil))/2.0
if Print:
    print(h_noncon)

wfn_spin.set_wfn(strategy='random')
if Print:
    wfn_spin.print_wfn()

spin_evolved = wfn_spin.time_evolve(time, h_noncon)
if Print:
    spin_evolved.print_wfn()

Exact Evolution Implementation of Diagonal Coulomb terms

In [ ]:
norb = 4
wfn = fqe.Wavefunction([[5, 1, norb]])
vij = numpy.zeros((norb, norb, norb, norb), dtype=numpy.complex128)
for i in range(norb):
            for j in range(norb):
                vij[i, j] += 4*(i % norb + 1)*(j % norb + 1)*0.21
                
wfn.set_wfn(strategy='random')

if Print:
    wfn.print_wfn()
    
hamil = fqe.get_diagonalcoulomb_hamiltonian(vij)
    
evolved = wfn.time_evolve(time, hamil)
if Print:
    evolved.print_wfn()

Exact evolution of individual n-body anti-Hermitian gnerators

In [ ]:
norb = 3
nele = 4
ops = FermionOperator('5^ 1^ 2 0', 3.0 - 1.j)
ops += FermionOperator('0^ 2^ 1 5', 3.0 + 1.j)
wfn = fqe.get_number_conserving_wavefunction(nele, norb)
wfn.set_wfn(strategy='random')
wfn.normalize()
if Print:
    wfn.print_wfn()
evolved = wfn.time_evolve(time, ops)
if Print:
    evolved.print_wfn()


 Approximate evolution of sums of n-body generators

Approximate evolution can be done for dense operators.

In [ ]:
lih_evolved = lihwfn.apply_generated_unitary(time, 'taylor', lih_hamiltonian, accuracy=1.e-8)
if Print:
    lih_evolved.print_wfn()

In [ ]:
norb = 2
nalpha = 1
nbeta = 1
nele = nalpha + nbeta
time = 0.05
h1e = numpy.zeros((norb*2, norb*2), dtype=numpy.complex128)
for i in range(2*norb):
    for j in range(2*norb):
        h1e[i, j] += (i+j) * 0.02
    h1e[i, i] += i * 2.0
hamil = fqe.get_general_hamiltonian((h1e,))
spec_lim = [-1.13199078e-03, 6.12720338e+00]
wfn = fqe.Wavefunction([[nele, nalpha - nbeta, norb]])
wfn.set_wfn(strategy='random')
if Print:
    wfn.print_wfn()
evol_wfn = wfn.apply_generated_unitary(time, 'chebyshev', hamil, spec_lim=spec_lim)
if Print:
    evol_wfn.print_wfn()

API for determining desired expectation values

In [ ]:
rdm1 = lihwfn.expectationValue('i^ j')
if Print:
    print(rdm1)
val = lihwfn.expectationValue('5^ 3')
if Print:
    print(2.*val)
trdm1 = fqe.expectationValue(lih_evolved, 'i j^', lihwfn)
if Print:
    print(trdm1)
val = fqe.expectationValue(lih_evolved, '5 3^', lihwfn)
if Print:
    print(2*val)

2.B.1 RDMs 
In addition to the above API higher order density matrices in addition to hole densities can be calculated.

In [ ]:
rdm2 = lihwfn.expectationValue('i^ j k l^')
if Print:
    print(rdm2)
rdm2 = fqe.expectationValue(lihwfn, 'i^ j^ k l', lihwfn)
if Print:
    print(rdm2)

2.B.2 Hamiltonian expectations (or any expectation values)

In [ ]:
li_h_energy = lihwfn.expectationValue(lih_hamiltonian)
if Print:
    print(li_h_energy)
li_h_energy = fqe.expectationValue(lihwfn, lih_hamiltonian, lihwfn)
if Print:
    print(li_h_energy)

2.B.3 Symmetry operations

In [ ]:
op = fqe.get_s2_operator()
print(lihwfn.expectationValue(op))
op = fqe.get_sz_operator()
print(lihwfn.expectationValue(op))
op = fqe.get_time_reversal_operator()
print(lihwfn.expectationValue(op))
op = fqe.get_number_operator()
print(lihwfn.expectationValue(op))
